In [1]:
import json
import torch
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import datasets
from peft import get_peft_model
from peft import LoraConfig
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer
from transformers import TrainingArguments

/home/pai/envs/test_qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 训练集共1949972条，只取前1000条
with open("medical_zh/train_zh_0.jsonl", "r") as f:
    lst = [json.loads(next(f)) for _ in range(1000)]
lst[0]

{'instruction': '血热的临床表现是什么?',
 'input': '',
 'output': '初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。'}

In [3]:
with open("train_lora.json", "w") as f:
    json.dump(lst, f, ensure_ascii=False)  # 有汉字，加ensure_ascii=False

In [4]:
# 加载Dataset。原文经过DataFrame中转，比较麻烦
ds = datasets.load_dataset("json", data_files="train_lora.json", split="train")  # 不加split会返回DatasetDict
ds[0]

Generating train split: 1000 examples [00:00, 31822.76 examples/s]


{'input': '',
 'instruction': '血热的临床表现是什么?',
 'output': '初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。'}

In [5]:
CKPT_PATH = "Qwen-1_8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(CKPT_PATH, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eod_id
tokenizer

QWenTokenizer(name_or_path='Qwen-1_8B-Chat', vocab_size=151851, model_max_length=8192, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [6]:
def process_func(example):
    MAX_LENGTH = 384

    instruction = tokenizer(
        "<|im_start|>system\n" +
        "你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>\n" +
        "<|im_start|>user\n" +
        example["instruction"] + example["input"] + "<|im_end|>\n"
    )
    response = tokenizer(
        "<|im_start|>assistant\n" +
        example["output"] + "<|im_end|>\n"
    )
    
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eod_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eod_id]

    # 截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
# 类似DataFrame的用法，并行处理数据
train_dataset = ds.map(process_func, remove_columns=ds.column_names)  # 去掉原始的列
train_dataset

Map: 100%|██████████| 1000/1000 [00:00<00:00, 1889.33 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [8]:
tokenizer.decode(train_dataset[1]['input_ids'])

'<|im_start|>system\n你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>\n<|im_start|>user\n帕金森叠加综合征的辅助治疗有些什么？<|im_end|>\n<|im_start|>assistant\n综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗<|im_end|>\n<|endoftext|>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, train_dataset[1]["labels"])))

'<|im_start|>assistant\n综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗<|im_end|>\n<|endoftext|>'

In [10]:
model = AutoModelForCausalLM.from_pretrained(CKPT_PATH, trust_remote_code=True, 
                                             torch_dtype=torch.bfloat16,
                                             # load_in_8bit=True
                                            )
model

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.14it/s]


QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-23): 24 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear(in_features=2048, out_features=5504, bias=False)
          (w2): Linear(in_features=2048, out_features=5504, bias=False)
          (c_proj): Linear(in_features=5504, out_features=2048, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=151936, bias=False)
)

In [11]:
model.dtype

torch.bfloat16

In [12]:
# 使用gradient_checkpointing+peft需要加这句补丁
model.enable_input_require_grads()

In [13]:
config = LoraConfig(
    # task_type=TaskType.CAUSAL_LM, # 可以不写
    target_modules=["c_attn", "c_proj", "w1", "w2"],  # 必须指定，Qwen官方的微调脚本也是写这几个
    # 原文说这三个参数比较通用，先留着
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'c_attn', 'w2', 'c_proj', 'w1'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, config)
model

PeftModel(
  (base_model): LoraModel(
    (model): QWenLMHeadModel(
      (transformer): QWenModel(
        (wte): Embedding(151936, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (rotary_emb): RotaryEmbedding()
        (h): ModuleList(
          (0-23): 24 x QWenBlock(
            (ln_1): RMSNorm()
            (attn): QWenAttention(
              (c_attn): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
            

In [15]:
model.print_trainable_parameters()

trainable params: 6,709,248 || all params: 1,843,537,920 || trainable%: 0.3639


In [16]:
# 定义训练配置
args = TrainingArguments(
    output_dir=f"./output/{CKPT_PATH}_flash_attn_v1",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 累计16个样本才迭代一次，1个epoch有1000/16=62步
    logging_steps=10,
    num_train_epochs=12,
    # gradient_checkpointing=True,  # 开启激活重计算，时间换空间
    save_steps=186,  # 也就是正好3个epoch保存一次
    learning_rate=1e-4
)

In [17]:
# 定义trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    # 用pad填充batch的三个字段
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)  # 原文设置padding=True，是默认值，省略掉
)

/home/pai/envs/test_qwen/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
# 训练前看看效果
response, history = model.eval().chat(tokenizer, "帕金森叠加综合征的辅助治疗有些什么?", history=None,
                                      system="你是一个医学助手，需要回答用户关于医学的问题：")
response, history

('帕金森叠加综合征是一种常见的神经系统退行性疾病，辅助治疗可以包括药物治疗、物理疗法和心理疗法。药物治疗主要包括抗帕金森病药、多巴胺受体激动剂等，有助于改善症状；物理疗法可以帮助患者增强肌肉力量，改善平衡能力，减轻关节疼痛；心理疗法可以帮助患者提高生活满意度，减轻焦虑和抑郁症状，从而改善生活质量。此外，还应定期进行康复训练，以进一步提高患者的日常生活能力和应对疾病的能力。',
 [('帕金森叠加综合征的辅助治疗有些什么?',
   '帕金森叠加综合征是一种常见的神经系统退行性疾病，辅助治疗可以包括药物治疗、物理疗法和心理疗法。药物治疗主要包括抗帕金森病药、多巴胺受体激动剂等，有助于改善症状；物理疗法可以帮助患者增强肌肉力量，改善平衡能力，减轻关节疼痛；心理疗法可以帮助患者提高生活满意度，减轻焦虑和抑郁症状，从而改善生活质量。此外，还应定期进行康复训练，以进一步提高患者的日常生活能力和应对疾病的能力。')])

In [19]:
trainer.train()

Step,Training Loss
10,2.514100
20,2.062700
30,2.151200
40,1.997000
50,2.037900
60,2.020600
70,1.900600
80,1.911300
90,1.880200
100,1.895800


Checkpoint destination directory ./output/Qwen-1_8B-Chat_new/checkpoint-186 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./output/Qwen-1_8B-Chat_new/checkpoint-372 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./output/Qwen-1_8B-Chat_new/checkpoint-558 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./output/Qwen-1_8B-Chat_new/checkpoint-744 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=744, training_loss=1.4083549335438719, metrics={'train_runtime': 825.8399, 'train_samples_per_second': 14.531, 'train_steps_per_second': 0.901, 'total_flos': 3.0783890072223744e+16, 'train_loss': 1.4083549335438719, 'epoch': 11.9})

In [20]:
# 训练后看看效果（用回训练集测试效果当然好）
response, history = model.eval().chat(tokenizer, "帕金森叠加综合征的辅助治疗有些什么?", history=None,
                                      system="你是一个医学助手，需要回答用户关于医学的问题：")
response, history

('低频重复经颅磁刺激疗法；穴位导入手法；康复训练；加药泵给药方案；综合护理干预；神经元激活治疗；补肾活血中药灌肠治疗；益智调神；细胞免疫调控技术；针对性功能锻炼指导；综合护理措施；舒适护理；营养脑细胞；改善微循环；心理护理；神经营养治疗；生物反馈联合功能锻炼指导；心理疏导；健康教育；饮食护理；个性化护理干预；早期综合治疗；以运动障碍为中心的护理策略；康复训练干预；结合康复训练；系统化护理干预；药物合理使用；延续性护理干预；系统性康复训练；语言言语干预治疗；指导患者功能锻炼；提高患者的活动能力；益肾培土法；功能锻炼指导；对症支持治疗；扶正固本；生活技能训练；认知行为治疗；康复干预；整体护理方法；鼓励患者主动参与治疗及护理；针刺百会、悬钟等穴；运动障碍的护理；蒙医护理干预；联合心理护理；功能锻炼；综合性护理；优质护理干预；护理干预；氧疗；心理护理指导；言语功能锻炼；加强肌张力训练指导；中西医结合治疗；低通气模式；介入治疗；综合治疗；改变不良生活习惯；补虚健脑；补虚为君；目的:治疗帕金森叠加综合征;心理指导;早期康复干预;综合护理干预措施；早期干预；高压氧舱吸入；语言功能训练；扩血管；临床护理路径式干预；心理指导与服务；配合康复锻炼指导；心理护理；人性化护理干预；家庭护理干预；环境创设；常规治疗；康复训练联合心理指导',
 [('帕金森叠加综合征的辅助治疗有些什么?',
   '低频重复经颅磁刺激疗法；穴位导入手法；康复训练；加药泵给药方案；综合护理干预；神经元激活治疗；补肾活血中药灌肠治疗；益智调神；细胞免疫调控技术；针对性功能锻炼指导；综合护理措施；舒适护理；营养脑细胞；改善微循环；心理护理；神经营养治疗；生物反馈联合功能锻炼指导；心理疏导；健康教育；饮食护理；个性化护理干预；早期综合治疗；以运动障碍为中心的护理策略；康复训练干预；结合康复训练；系统化护理干预；药物合理使用；延续性护理干预；系统性康复训练；语言言语干预治疗；指导患者功能锻炼；提高患者的活动能力；益肾培土法；功能锻炼指导；对症支持治疗；扶正固本；生活技能训练；认知行为治疗；康复干预；整体护理方法；鼓励患者主动参与治疗及护理；针刺百会、悬钟等穴；运动障碍的护理；蒙医护理干预；联合心理护理；功能锻炼；综合性护理；优质护理干预；护理干预；氧疗；心理护理指导；言语功能锻炼；加强肌张力训练指导；中西医结合治疗；